## Web Scraping des Données d'Électroménager sur Amazon

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import re

def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Exécution en arrière-plan
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

def scrape_amazon_product(driver, categorie, max_pages=2):
    base_url = "https://www.amazon.fr"
    products = []
    
    for page in range(1, max_pages + 1):
        url = f"{base_url}/s?k={categorie.replace(' ', '+')}&page={page}"
        print(f"Scraping page {page}: {url}")
        
        driver.get(url)
        time.sleep(2)  # Attendre le chargement
        
        # Accepter les cookies si nécessaire
        try:
            cookie_button = driver.find_element(By.ID, "sp-cc-accept")
            cookie_button.click()
            time.sleep(1)
        except:
            pass
        
        items = driver.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')
        
        for item in items:
            try:
                # Nom du produit
                nom = item.find_element(By.TAG_NAME, "h2").text.strip()
                
                # Prix
                try:
                    prix_whole = item.find_element(By.CLASS_NAME, "a-price-whole").text
                    prix_fraction = item.find_element(By.CLASS_NAME, "a-price-fraction").text
                    prix = float(f"{prix_whole}{prix_fraction}".replace(",", "."))
                except:
                    prix = None
                
                # Note client
                try:
                    note = item.find_element(By.CLASS_NAME, "a-icon-alt").get_attribute("innerHTML")
                    note = float(note.split()[0].replace(",", "."))
                except:
                    note = None
                
                # Nombre d'évaluations
                try:
                    evaluations = item.find_element(By.XPATH, './/span[@class="a-size-base s-underline-text"]').text
                    evaluations = int(evaluations.replace("(", "").replace(")", "").replace(",", "").replace(".", ""))
                except:
                    evaluations = 0
                
                # Livraison
                try:
                    livraison = item.find_element(By.XPATH, './/div[contains(@class, "s-align-children-center")]').text.strip()
                except:
                    livraison = "N/A"
                
                # Quantité vendue (approximation)
                quantite_vendue = evaluations * 100  # Estimation grossière
                
                products.append({
                    'categorie': categorie,
                    'enseigne': 'Amazon',
                    'nom': nom,
                    'prix': prix,
                    'note_client': note,
                    'nombre_avis': evaluations,
                    'livraison': livraison,
                    'quantite_vendue': quantite_vendue
                })
                
            except Exception as e:
                print(f"Erreur sur un produit: {str(e)}")
                continue
    
    return products

def main():
    driver = setup_driver()
    
    categories = [
        "machine à laver",
        "réfrigérateur",
        "lave-vaisselle",
        "four",
        "micro-ondes"
    ]
    
    all_products = []
    
    for categorie in categories:
        print(f"\nScraping catégorie: {categorie}")
        products = scrape_amazon_product(driver, categorie)
        all_products.extend(products)
        time.sleep(5)  # Délai entre les catégories
    
    driver.quit()
    
    df = pd.DataFrame(all_products)
    df.to_csv('amazon_electromenager_selenium.csv', index=False)
    print("\nScraping terminé. Données sauvegardées dans amazon_electromenager_selenium.csv")

if __name__ == "__main__":
    main()


Scraping catégorie: machine à laver
Scraping page 1: https://www.amazon.fr/s?k=machine+à+laver&page=1
Scraping page 2: https://www.amazon.fr/s?k=machine+à+laver&page=2

Scraping catégorie: réfrigérateur
Scraping page 1: https://www.amazon.fr/s?k=réfrigérateur&page=1
Scraping page 2: https://www.amazon.fr/s?k=réfrigérateur&page=2

Scraping catégorie: lave-vaisselle
Scraping page 1: https://www.amazon.fr/s?k=lave-vaisselle&page=1
Scraping page 2: https://www.amazon.fr/s?k=lave-vaisselle&page=2

Scraping catégorie: four
Scraping page 1: https://www.amazon.fr/s?k=four&page=1
Scraping page 2: https://www.amazon.fr/s?k=four&page=2

Scraping catégorie: micro-ondes
Scraping page 1: https://www.amazon.fr/s?k=micro-ondes&page=1
Scraping page 2: https://www.amazon.fr/s?k=micro-ondes&page=2

Scraping terminé. Données sauvegardées dans amazon_electromenager_selenium.csv


In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import re

def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

def scrape_amazon_tv(driver, max_pages=2):
    base_url = "https://www.amazon.fr"
    category = "téléviseur"
    products = []
    
    for page in range(1, max_pages + 1):
        url = f"{base_url}/s?k={category.replace(' ', '+')}&page={page}"
        print(f"Scraping page {page}: {url}")
        
        driver.get(url)
        time.sleep(3)  # Attendre le chargement
        
        # Accepter les cookies si nécessaire
        try:
            cookie_button = driver.find_element(By.ID, "sp-cc-accept")
            cookie_button.click()
            time.sleep(1)
        except:
            pass
        
        items = driver.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')
        
        for item in items:
            try:
                # Nom du produit
                nom = item.find_element(By.TAG_NAME, "h2").text.strip()
                
                # Extraire les spécifications TV
                taille = re.search(r'(\d+["’’])', nom)
                technologie = re.search(r'(OLED|QLED|LED|NanoCell|Mini-LED)', nom, re.IGNORECASE)
                resolution = re.search(r'(HD|Full HD|4K|8K|UHD)', nom, re.IGNORECASE)
                marque = nom.split()[0] if len(nom.split()) > 0 else "Inconnue"
                
                # Prix
                try:
                    prix_whole = item.find_element(By.CLASS_NAME, "a-price-whole").text
                    prix_fraction = item.find_element(By.CLASS_NAME, "a-price-fraction").text
                    prix = float(f"{prix_whole}{prix_fraction}".replace(",", "."))
                except:
                    prix = None
                
                # Note client
                try:
                    note = item.find_element(By.CLASS_NAME, "a-icon-alt").get_attribute("innerHTML")
                    note = float(note.split()[0].replace(",", "."))
                except:
                    note = None
                
                # Nombre d'évaluations
                try:
                    evaluations = item.find_element(By.XPATH, './/span[@class="a-size-base s-underline-text"]').text
                    evaluations = int(evaluations.replace("(", "").replace(")", "").replace(",", "").replace(".", ""))
                except:
                    evaluations = 0
                
                # Livraison
                try:
                    livraison = item.find_element(By.XPATH, './/div[contains(@class, "s-align-children-center")]').text.strip()
                except:
                    livraison = "N/A"
                
                # Quantité vendue (estimation adaptée pour les TV)
                quantite_vendue = evaluations * 60  # Estimation spécifique pour les TV
                
                products.append({
                    'categorie': 'Téléviseur',
                    'enseigne': 'Amazon',
                    'marque': marque,
                    'nom': nom,
                    'taille_ecran': taille.group(1) if taille else "N/A",
                    'technologie': technologie.group(1) if technologie else "N/A",
                    'resolution': resolution.group(1) if resolution else "N/A",
                    'prix': prix,
                    'note_client': note,
                    'nombre_avis': evaluations,
                    'livraison': livraison,
                    'quantite_vendue': quantite_vendue
                })
                
            except Exception as e:
                print(f"Erreur sur un produit: {str(e)}")
                continue
    
    return products

def main():
    driver = setup_driver()
    
    try:
        print("\nScraping des téléviseurs sur Amazon...")
        tvs = scrape_amazon_tv(driver)
        
        df = pd.DataFrame(tvs)
        
        # Réorganisation des colonnes pour les TV
        colonnes = ['categorie', 'enseigne', 'marque', 'nom', 'taille_ecran', 
                   'technologie', 'resolution', 'prix', 'note_client', 
                   'nombre_avis', 'quantite_vendue', 'livraison']
        df = df[colonnes]
        
        df.to_csv('amazon_tv_selenium.csv', index=False)
        print("\nScraping terminé. Données sauvegardées dans amazon_tv_selenium.csv")
        
        # Aperçu des données
        print("\nAperçu des données scrapées:")
        print(df.head())
        
    finally:
        driver.quit()

if __name__ == "__main__":
    main()


Scraping des téléviseurs sur Amazon...
Scraping page 1: https://www.amazon.fr/s?k=téléviseur&page=1
Scraping page 2: https://www.amazon.fr/s?k=téléviseur&page=2

Scraping terminé. Données sauvegardées dans amazon_tv_selenium.csv

Aperçu des données scrapées:
    categorie enseigne   marque  \
0  Téléviseur   Amazon  THOMSON   
1  Téléviseur   Amazon  THOMSON   
2  Téléviseur   Amazon     Sony   
3  Téléviseur   Amazon     CHIQ   
4  Téléviseur   Amazon      RCA   

                                                 nom taille_ecran technologie  \
0  THOMSON 32'' (80 cm) HD - LED Fire TV - Comman...          N/A         LED   
1  THOMSON – 50 Pouces (126 cm) Ultra HD Blanc – ...          N/A         N/A   
2  Sony TV Bravia KD-50X75WL : TV 4K Ultra HD LED...          N/A         LED   
3  CHIQ L32G7L,32 Pouces, Télévision Android 11 B...          N/A         N/A   
4  RCA TV 32 Pouces (80 cm) sans Bord LED HD Télé...          N/A         LED   

  resolution     prix  note_client  nombre

In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import re

def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

def scrape_amazon_smartphones(driver, max_pages=2):
    base_url = "https://www.amazon.fr"
    category = "smartphone"
    products = []
    
    for page in range(1, max_pages + 1):
        url = f"{base_url}/s?k={category.replace(' ', '+')}&page={page}"
        print(f"Scraping page {page}: {url}")
        
        driver.get(url)
        time.sleep(3)  # Attendre le chargement
        
        # Accepter les cookies si nécessaire
        try:
            cookie_button = driver.find_element(By.ID, "sp-cc-accept")
            cookie_button.click()
            time.sleep(1)
        except:
            pass
        
        items = driver.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')
        
        for item in items:
            try:
                # Nom du produit
                nom = item.find_element(By.TAG_NAME, "h2").text.strip()
                
                # Extraire les spécifications smartphones
                marque = nom.split()[0] if len(nom.split()) > 0 else "Inconnue"
                modele = re.search(r'(Pro|Max|Plus|Mini|Lite|Ultra|SE|Note|Edge|Galaxy|iPhone)', nom, re.IGNORECASE)
                systeme = "iOS" if "iPhone" in nom else "Android"
                ram = re.search(r'(\d+GB)\sRAM', nom, re.IGNORECASE)
                stockage = re.search(r'(\d+GB|\d+TB)\s(?!RAM)', nom, re.IGNORECASE)
                reseau = "5G" if "5G" in nom else "4G"
                
                # Prix
                try:
                    prix_whole = item.find_element(By.CLASS_NAME, "a-price-whole").text
                    prix_fraction = item.find_element(By.CLASS_NAME, "a-price-fraction").text
                    prix = float(f"{prix_whole}{prix_fraction}".replace(",", "."))
                except:
                    prix = None
                
                # Note client
                try:
                    note = item.find_element(By.CLASS_NAME, "a-icon-alt").get_attribute("innerHTML")
                    note = float(note.split()[0].replace(",", "."))
                except:
                    note = None
                
                # Nombre d'évaluations
                try:
                    evaluations = item.find_element(By.XPATH, './/span[@class="a-size-base s-underline-text"]').text
                    evaluations = int(evaluations.replace("(", "").replace(")", "").replace(",", "").replace(".", ""))
                except:
                    evaluations = 0
                
                # Livraison
                try:
                    livraison = item.find_element(By.XPATH, './/div[contains(@class, "s-align-children-center")]').text.strip()
                except:
                    livraison = "N/A"
                
                # Quantité vendue (estimation adaptée pour smartphones)
                quantite_vendue = evaluations * 80  # Coefficient plus élevé pour smartphones
                
                products.append({
                    'categorie': 'Smartphone',
                    'enseigne': 'Amazon',
                    'marque': marque,
                    'modele': modele.group(1) if modele else "Standard",
                    'systeme': systeme,
                    'ram': ram.group(1) if ram else "N/A",
                    'stockage': stockage.group(1) if stockage else "N/A",
                    'reseau': reseau,
                    'nom_complet': nom,
                    'prix': prix,
                    'note_client': note,
                    'nombre_avis': evaluations,
                    'livraison': livraison,
                    'quantite_vendue': quantite_vendue
                })
                
            except Exception as e:
                print(f"Erreur sur un produit: {str(e)}")
                continue
    
    return products

def main():
    driver = setup_driver()
    
    try:
        print("\nScraping des smartphones sur Amazon...")
        smartphones = scrape_amazon_smartphones(driver)
        
        df = pd.DataFrame(smartphones)
        
        # Réorganisation des colonnes pour smartphones
        colonnes = ['categorie', 'enseigne', 'marque', 'modele', 'systeme', 
                   'ram', 'stockage', 'reseau', 'nom_complet', 'prix', 
                   'note_client', 'nombre_avis', 'quantite_vendue', 'livraison']
        df = df[colonnes]
        
        df.to_csv('amazon_smartphones_selenium.csv', index=False)
        print("\nScraping terminé. Données sauvegardées dans amazon_smartphones_selenium.csv")
        
        # Aperçu des données
        print("\nAperçu des données scrapées:")
        print(df.head())
        
        # Statistiques basiques
        print("\nStatistiques:")
        print(f"- Nombre de smartphones: {len(df)}")
        print(f"- Prix moyen: {df['prix'].mean():.2f}€")
        print(f"- Répartition par OS:\n{df['systeme'].value_counts()}")
        
    finally:
        driver.quit()

if __name__ == "__main__":
    main()


Scraping des smartphones sur Amazon...
Scraping page 1: https://www.amazon.fr/s?k=smartphone&page=1
Scraping page 2: https://www.amazon.fr/s?k=smartphone&page=2

Scraping terminé. Données sauvegardées dans amazon_smartphones_selenium.csv

Aperçu des données scrapées:
    categorie enseigne  marque    modele  systeme  ram stockage reseau  \
0  Smartphone   Amazon  Google       Pro  Android  N/A      N/A     4G   
1  Smartphone   Amazon  Google       Pro  Android  N/A      N/A     4G   
2  Smartphone   Amazon   HONOR  Standard  Android  N/A      N/A     4G   
3  Smartphone   Amazon  DOOGEE  Standard  Android  N/A      N/A     4G   
4  Smartphone   Amazon  Xiaomi  Standard  Android  4GB    128GB     4G   

                                         nom_complet     prix  note_client  \
0  Google Pixel 9 Pro – Smartphone Android débloq...  75500.0          4.3   
1  Google Pixel 9 Pro XL – Smartphone Android déb...  91345.0          4.4   
2  HONOR X6b Smartphone 4Go 128Go, 5200mAh Batter...